In [ ]:
import numpy as np
from collections import deque
import random

import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

#Игра

In [ ]:
# Игра крестики-нолики
class TicTacToe:
    def __init__(self, player_1, player_2, board_size=3, win_size=3):
        self.players = {-1: player_1,
                         1: player_2}

        self.wins = {player_1.name: 0,
                     player_2.name: 0}

        self.board_size=board_size
        self.win_size = win_size
        self._kernel = self._create_kernel()


    # Создает ядро свертки для расчета побед
    def _create_kernel(self):
        kernel = np.zeros((2 * self.win_size + 2, self.win_size, self.win_size))
        for i in range(self.win_size):
            kernel[i, i, :] = np.ones(self.win_size)
        for i in range(self.win_size, 2 * self.win_size):
            kernel[i, :, i - self.win_size] = np.ones(self.win_size).T
        kernel[2 * self.win_size] = np.eye(self.win_size)
        kernel[2 * self.win_size + 1] = np.fliplr(np.eye(self.win_size))
        return kernel


    # Проверяет победы для состояний states, в кот. ходы были совершены игроками turns, turn={-1, 1}
    def _test_win(self, state, turn):
        rows, cols, w_size = *state.shape, self.win_size
        expanded_states = np.lib.stride_tricks.as_strided(
            state,
            shape=(rows - w_size + 1, cols - w_size + 1, w_size, w_size),
            strides=(*state.strides, *state.strides),
            writeable=False,
        )
        feature_map = np.einsum('xyij,sij->sxy', expanded_states, self._kernel)
        return -turn * (feature_map == turn * w_size).any().astype(int)


    # Проигрывание нескольких полных эпизодов
    def play(self, num_games=1, visualize=False):
        transitions = []
        for t in range(num_games):
            next_turn = turn = -1
            state = (np.zeros((self.board_size, self.board_size)), turn) # Начальное состояние игры. state = (state2d, turn)
            if visualize:
                self.visualize_state(state, turn)
            while(next_turn != 0):
                state_2d, turn = state
                current_player = self.players[turn]
                action = current_player.get_action(state)
                next_state_2d, next_turn, reward = self.play_turn(state, action)
                transitions.append((turn * state_2d, action, reward, -turn * next_state_2d, next_turn == 0))   #state, action, reward, new_state, done
                if visualize:
                    self.visualize_state((next_state_2d, next_turn), turn)
                if next_turn == 0:
                    if visualize:
                        if (reward == 0): print('Ничья!\n')
                        else: print(f'Победа ({self.players[reward * turn].name})!\n')
                    if reward != 0:
                        self.wins[self.players[reward * turn].name] += 1
                    self.players = {-1: self.players[1], 1: self.players[-1]}
                state = next_state_2d, next_turn
        return transitions


    # Выполнение хода и проверка на некорректный ход (проигрышь) / выигрыш / ничью
    def play_turn(self, state, action): # next_state2d, next_turn, reward
        state2d, turn = state
        next_state2d = state2d.copy()

        # Проверка корректности хода
        if (state2d[(action)] != 0):
            return next_state2d, 0, -1        # Игрок проиграл (# next_turn == 0 => Игра окончена)

        # Совершение хода
        next_state2d[action] = turn

        # Проверка победы
        if self._test_win(next_state2d, turn):
            return next_state2d, 0, 1         # Текущий игрок побеждает (next_turn == 0 => Игра окончена)

        # Проверка ничьи
        if (next_state2d != 0).all():
            return next_state2d, 0, 0         # Ничья (next_turn == 0 => Игра окончена)

        # Инчае, ход следующего игрока
        return next_state2d, -turn, 0         # next_turn == -turn => Смена хода


    # Выводит на экран состояние игры после хода игрока
    @staticmethod
    def visualize_state(next_state, turn):
        next_state2d, next_turn = next_state
        print(f"player {turn}'s turn:")
        if (next_state2d == 0).all() and turn == 0:
            print("[invalid state]\n\n")
        else:
            print(str(next_state2d)
                  .replace(".", "")
                  .replace("[[", "")
                  .replace(" [", "")
                  .replace("]]", "")
                  .replace("]", "")
                  .replace("-0", " .")
                  .replace("0", ".")
                  .replace("-1", " X")
                  .replace("1", "O")
            )


    @staticmethod
    def print_transitions(transitions):
        states, actions, rewards, next_states, dones = zip(*transitions)
        for i in np.arange(len(states)):
            print("\033[31m{}.".format(i + 1), '\033[30m')
            TicTacToe.visualize_state((next_states[i], -1), 1)
            print('\naction = ', actions[i] + np.array([1, 1]), end='\n')
            print('reward = ', rewards[i], end='\n')
            if (dones[i]): print('Игра окончена', end='\n\n')
            else: print('Игра продолжается', end='\n\n')

#Игроки

In [ ]:
class Human:
    def __init__(self, name='Human'):
        self.name = name

    def get_action(self, state):
        state2d, turn = state
        print('Введите ваш ход (Строка, столбец)')
        row, col = map(int, input().split())
        while (state2d[row - 1, col - 1] != 0):
            print('Клетка занята!')
            print('Введите ваш ход (Строка, столбец)')
            row, col = map(int, input().split())
        return row - 1, col - 1

In [ ]:
# Игрок Рандом с преимуществами:
# 1. Если есть возможность выиграть за один ход, он делает это (win = True)
# 2. Если у соперника есть возможность выиграть в следующем ходу, он блокирует этот ход (defense = True)
# 3. Если есть возможность построить четверку, он делает это (win_2 = True)
# 4. Если у соперника есть возможность построить четверку в следующем ходу, он блокирует этот ход (defense_2 = True)
# 5. Иначе, выбирает случайный ход из множества допустимых
class RandomPlus:
    def __init__(self, board_size=3, win_size=3, name='RandomPlus',
                 win=False, defense=False, win_2=False, defense_2=False):
        self.name = name
        self.board_size = board_size
        self.win_size = win_size

        self.win = win
        self.defense = defense

        self.win_2 = win_2
        self.defense_2 = defense_2

        if win or defense:
            self._kernel = self._create_kernel(win_size)

        if win_2 or defense_2:
            self._kernel_2 = self._create_kernel(win_size - 1)


    # Создает ядро свертки для расчета потенциальных побед
    def _create_kernel(self, win_size):
        kernel = np.zeros((2 * win_size + 2, win_size, win_size))
        for i in range(win_size):
            kernel[i, i, :] = np.ones(win_size)
        for i in range(win_size, 2 * win_size):
            kernel[i, :, i - win_size] = np.ones(win_size).T
        kernel[2 * win_size] = np.eye(win_size)
        kernel[2 * win_size + 1] = np.fliplr(np.eye(win_size))
        return kernel


    def get_action(self, state):
        state2d, turn = state
        rows, cols, w_size = *state2d.shape, self.win_size

        if self.win or self.defense:
            expanded_states = np.lib.stride_tricks.as_strided(
                state2d,
                shape=(rows - w_size + 1, cols - w_size + 1, w_size, w_size),
                strides=(*state2d.strides, *state2d.strides),
                writeable=False,
            )
            feature_map = np.einsum('xyij,sij->sxy', expanded_states, self._kernel)

            if self.win:
                wins = np.array(np.where(turn * feature_map == w_size - 1))
                if wins.shape[1] > 0:
                    index = np.random.randint(0, wins.shape[1])
                    K, I, J = wins[:, index]
                    indxs = np.where(np.logical_and((self._kernel[K] == 1), (state2d[I: I + w_size, J: J + w_size] == 0)))
                    return tuple(np.array(indxs)[:, 0] + [I, J])

            if self.defense:
                defenses = np.array(np.where(-turn * feature_map == w_size - 1))
                if defenses.shape[1] > 0:
                    index = np.random.randint(0, defenses.shape[1])
                    K, I, J = defenses[:, index]
                    indxs = np.where(np.logical_and((self._kernel[K] == 1), (state2d[I: I + w_size, J: J + w_size] == 0)))
                    return tuple(np.array(indxs)[:, 0] + [I, J])

        if self.win_2 or self.defense_2:
            expanded_states = np.lib.stride_tricks.as_strided(
                state2d,
                shape=(rows - w_size + 2, cols - w_size + 2, w_size - 1, w_size - 1),
                strides=(*state2d.strides, *state2d.strides),
                writeable=False,
            )
            feature_map = np.einsum('xyij,sij->sxy', expanded_states, self._kernel_2)

            if self.win_2:
                wins = np.array(np.where(turn * feature_map == w_size - 2))
                if wins.shape[1] > 0:
                    index = np.random.randint(0, wins.shape[1])
                    K, I, J = wins[:, index]
                    indxs = np.where(np.logical_and((self._kernel_2[K] == 1), (state2d[I: I + w_size - 1, J: J + w_size - 1] == 0)))
                    return tuple(np.array(indxs)[:, 0] + [I, J])

            if self.defense_2:
                defenses = np.array(np.where(-turn * feature_map == w_size - 2))
                if defenses.shape[1] > 0:
                    index = np.random.randint(0, defenses.shape[1])
                    K, I, J = defenses[:, index]
                    indxs = np.where(np.logical_and((self._kernel_2[K] == 1), (state2d[I: I + w_size - 1, J: J + w_size - 1] == 0)))
                    return tuple(np.array(indxs)[:, 0] + [I, J])

        zero_idxs = np.argwhere(state2d == 0)
        return tuple(zero_idxs[np.random.randint(len(zero_idxs))])

In [ ]:
class DQNAgent(nn.Module):
    def __init__(self, epsilon=0, name='DQNAgent', masking=False):
        super().__init__()

        self.name = name
        self.epsilon = epsilon
        self.n_channels = 3
        self.masking = masking    # Маскирование (ВКЛЮЧАТЬ ТОЛЬКО ПРИ ИНФЕРЕНСЕ)

        self.network = nn.Sequential(
            nn.Conv2d(self.n_channels, 128, kernel_size=(3, 3), padding='same'),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=(3, 3), padding='same'),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=(3, 3), padding='same'),
            nn.ReLU(),
            nn.Conv2d(256, 128, kernel_size=(3, 3), padding='same'),
            nn.ReLU(),
            nn.Conv2d(128, 1, kernel_size=(3, 3), padding='same')
        )

    def forward(self, x):
        x = torch.stack([x == 1, x == -1, x == 0], axis=1).float()
        return self.network(x).squeeze(1)

    def greedy_action(self, state, device=device):
        state2d, turn = state
        state_t = torch.FloatTensor(turn * state2d).unsqueeze(0).to(device)
        q_values = self.forward(state_t).squeeze(0).detach().cpu().numpy()
        if self.masking:
            q_values[state2d != 0] = -float("Inf")
        return np.unravel_index(q_values.argmax(), q_values.shape)

    def random_action(self, state):
        state2d, turn = state
        zero_idxs = np.argwhere(state2d == 0)
        return tuple(zero_idxs[np.random.randint(len(zero_idxs))])

    def get_action(self, state):
        if random.random() < self.epsilon:
            action = self.random_action(state)
        else:
            action = self.greedy_action(state)
        return action

# Буферы

In [ ]:
# Обычный буфер
class ReplayBuffer(object):
    def __init__(self, size):
        self._storage = deque(maxlen=size)

    def __len__(self):
        return len(self._storage)

    def add(self, transition):
        self._storage.append(transition)

    def sample(self, batch_size, augmentation=False):
        batch = random.sample(self._storage, batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)
        states, actions, rewards, next_states, dones = np.array(states), np.array(actions), np.array(rewards), np.array(next_states), np.array(dones)

        if augmentation:
            # ======== ДЛЯ ВСЕГО БАТЧА ОДИНАКОВАЯ АУГМЕНТАЦИЯ ========
            # n = states.shape[-1] - 1
            # k = np.random.randint(0, 4)
            # states = np.rot90(states, k, axes=(1,2)).copy()
            # next_states = np.rot90(next_states, k, axes=(1,2)).copy()

            # i, j = actions[:, 0], actions[:, 1]
            # if k == 1: actions = np.column_stack((n - j, i))
            # if k == 2: actions = np.column_stack((n - i, n - j))
            # if k == 3: actions = np.column_stack((j, n - i))


            # ======== ДЛЯ КАЖДОГО ЭЛЕМЕНТА БАТЧА ОТДЕЛЬНО ========
            n = states.shape[-1] - 1
            k = np.random.randint(0, 4, size=batch_size)

            mask = [None] * 4
            for i in range(1, 4):
                mask[i] = k == i
                states[mask[i]] = np.rot90(states[mask[i]], i, axes=(1, 2))
                next_states[mask[i]] = np.rot90(next_states[mask[i]], i, axes=(1, 2))

            i, j = actions[:, 0], actions[:, 1]
            actions[mask[1]] = np.column_stack((n - j[mask[1]], i[mask[1]]))
            actions[mask[2]] = np.column_stack((n - i[mask[2]], n - j[mask[2]]))
            actions[mask[3]] = np.column_stack((j[mask[3]], n - i[mask[3]]))


            # ======== УВЕЛИЧЕНИЕ X4 ========
            # n = states.shape[-1] - 1
            # i, j = actions[:, 0], actions[:, 1]

            # states = np.concatenate([np.rot90(states, k, axes=(1, 2)) for k in range(4)], axis=0)
            # next_states = np.concatenate([np.rot90(next_states, k, axes=(1, 2)) for k in range(4)], axis=0)
            # actions = np.concatenate([actions,
            #                           np.column_stack((n - j, i)),
            #                           np.column_stack((n - i, n - j)),
            #                           np.column_stack((j, n - i))], axis=0)
            # rewards = np.tile(rewards, 4)
            # dones = np.tile(dones, 4)

        return states, actions, rewards, next_states, dones

In [ ]:
# =========== Prioritized Replay Buffer With Augmentation ===========
class PrioritizedBuffer(object):
    def __init__(self, capacity, prob_alpha=0.6):
        self.prob_alpha = prob_alpha
        self.capacity = capacity
        self.buffer = []
        self.pos = 0
        self.priorities = np.zeros((capacity,), dtype=np.float32)

    def add(self, state, action, reward, next_state, done):
        max_prio = self.priorities.max() if self.buffer else 1.0

        if len(self.buffer) < self.capacity:
            self.buffer.append((state, action, reward, next_state, done))
        else:
            self.buffer[self.pos] = (state, action, reward, next_state, done)

        self.priorities[self.pos] = max_prio
        self.pos = (self.pos + 1) % self.capacity

    def sample(self, batch_size, beta=0.4, augmentation=False):
        if len(self.buffer) == self.capacity:
            prios = self.priorities
        else:
            prios = self.priorities[:self.pos]

        probs  = prios ** self.prob_alpha
        probs /= probs.sum()

        indices = np.random.choice(len(self.buffer), batch_size, p=probs)
        samples = [self.buffer[idx] for idx in indices]

        total    = len(self.buffer)
        weights  = (total * probs[indices]) ** (-beta)
        weights /= weights.max()
        weights  = np.array(weights, dtype=np.float32)

        states, actions, rewards, next_states, dones = zip(*samples)
        states, actions, rewards, next_states, dones = np.array(states), np.array(actions), np.array(rewards), np.array(next_states), np.array(dones)

        if augmentation:
            n = states.shape[-1] - 1
            k = np.random.randint(0, 4, size=batch_size)

            mask = [None] * 4
            for i in range(1, 4):
                mask[i] = k == i
                states[mask[i]] = np.rot90(states[mask[i]], i, axes=(1, 2))
                next_states[mask[i]] = np.rot90(next_states[mask[i]], i, axes=(1, 2))

            i, j = actions[:, 0], actions[:, 1]
            actions[mask[1]] = np.column_stack((n - j[mask[1]], i[mask[1]]))
            actions[mask[2]] = np.column_stack((n - i[mask[2]], n - j[mask[2]]))
            actions[mask[3]] = np.column_stack((j[mask[3]], n - i[mask[3]]))

        return states, actions, rewards, next_states, dones, indices, weights

    def update_priorities(self, batch_indices, batch_priorities):
        for idx, prio in zip(batch_indices, batch_priorities):
            self.priorities[idx] = prio

    def __len__(self):
        return len(self.buffer)

#Функции и гиперпараметры для обучения

In [ ]:
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed);

In [ ]:
board_size = 7
win_size = 5

In [ ]:
# Гиперпараметры метода DQN

batch_size = 128        # 512 - много
total_steps = 90_000

decay_steps = 60_000
init_epsilon = 1
final_epsilon = 0.25     # 0.02 - мало; 0.1 - мало

loss_freq = 100
refresh_target_network_freq = 100    # 1000 - много, 50 - мало

eval_freq = 500
n_eval_games = 100

max_grad_norm = 50

gamma = 0.9

In [ ]:
agent = DQNAgent(init_epsilon).to(device)

target_network = DQNAgent(init_epsilon).to(device)
target_network.load_state_dict(agent.state_dict())

optimizer = torch.optim.Adam(agent.parameters(), lr=1e-4)
exp_replay = PrioritizedBuffer(16_000) #ReplayBuffer(16_000)

In [ ]:
sum([p.numel() for p in agent.parameters()])

1185025

In [ ]:
# Возвращает temporal difference loss
def compute_td_loss(states, actions, rewards, next_states, dones,
                    agent, target_network, weights=None, indices=None,    #exp_replay
                    gamma=0.9, device=device, prioritized=False):

    states = torch.tensor(states, device=device, dtype=torch.float32)                # shape: [batch_size, state_dim]
    actions = torch.tensor(actions, device=device, dtype=torch.int64)                # shape: [batch_size]
    rewards = torch.tensor(rewards, device=device, dtype=torch.float32)              # shape: [batch_size]
    next_states = torch.tensor(next_states, device=device, dtype=torch.float32)      # shape: [batch_size, state_dim]
    dones = torch.tensor(dones, device=device, dtype=torch.int64)                    # shape: [batch_size]

    predicted_qvalues = agent(states)                                                # shape: [batch_size, n_actions]
    predicted_next_qvalues = target_network(next_states)                             # shape: [batch_size, n_actions]
    predicted_qvalues_for_actions = predicted_qvalues[range(len(actions)), actions[:, 0], actions[:, 1]]  # shape: [batch_size]
    next_state_values = predicted_next_qvalues.view(dones.shape[0], -1).max(axis=1).values
    target_qvalues_for_actions = rewards - (1 - dones) * gamma * next_state_values

    if prioritized:   #[Prioterized DQN]
        weights = torch.tensor(weights, device=device, dtype=torch.float32)
        loss = weights * (predicted_qvalues_for_actions - target_qvalues_for_actions.detach()) ** 2
        prios = (loss + 1e-5).data.cpu().numpy()  # Обновление приоритетов
        loss = torch.mean(loss)
        exp_replay.update_priorities(indices, prios)
        return loss
    else:
        return torch.mean((predicted_qvalues_for_actions - target_qvalues_for_actions.detach()) ** 2)  #loss

# Рассчитывает epsilon на текущем шаге step
def linear_decay(init_epsilon, final_epsilon, step, decay_steps):
    return max(init_epsilon - step * (init_epsilon - final_epsilon) / decay_steps, final_epsilon)

# Обучение

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
main_random = RandomPlus(board_size, win_size, win=True)
game = TicTacToe(agent, main_random, board_size=board_size, win_size=win_size)

In [ ]:
PATH = f'/content/drive/MyDrive/TicTacToe_9/'

loss = None
loss_values = []
reward_values = []

In [ ]:
with open('out.txt', 'w') as f:
  for t in range(total_steps):
    print(f't = {t}. Ход {game.players[-1].name}', file=f)

    state = (np.zeros((board_size, board_size)), -1) # Начальное состояние игры. state = (state_2d, turn)
    turn = next_turn = -1

    while(next_turn != 0):
        current_player = game.players[turn]
        if current_player.name == 'DQNAgent':
            agent.epsilon = linear_decay(init_epsilon, final_epsilon, t, decay_steps)

        action = agent.get_action(state)
        print(action, file=f)
        next_state_2d, next_turn, reward = game.play_turn(state, action)
        state_2d, turn = state

        if next_turn == 0:
            if (reward == 0): print('Ничья!\n', file=f)
            else: print(f'Победа ({game.players[reward * turn].name})!\n', file=f)
            game.players = {-1: game.players[1], 1: game.players[-1]}

        exp_replay.add(turn * state_2d, action, reward, next_turn * next_state_2d, next_turn == 0) #state, action, reward, new_state, done

        # Обучение на минибатче
        if len(exp_replay) >= batch_size:
            states, actions, rewards, next_states, dones, indices, weights = exp_replay.sample(batch_size, augmentation=True)
            loss = compute_td_loss(states, actions, rewards, next_states, dones,
                                   agent, target_network, weights, indices, gamma, prioritized=True)
            loss.backward()
            grad_norm = nn.utils.clip_grad_norm_(agent.parameters(), max_grad_norm)
            optimizer.step()
            optimizer.zero_grad()

        state = next_state_2d, next_turn

    # Каждые refresh_target_network_freq обновляются веса target сети
    if t % refresh_target_network_freq == 0:
        target_network.load_state_dict(agent.state_dict())

    # Вывод лосса  с заданной частотой
    if t % loss_freq == 0 and t > 0:
        loss_values.append(loss)
        print(f"t = {str(t):5}\t loss = {loss}\t eps = {round(agent.epsilon, 4)}")

    # Вывод награды с заданной частотой
    if t % eval_freq == 0:
        agent.epsilon = 0
        eval_random = RandomPlus(board_size, win_size, win=True, defense=True)
        eval_game = TicTacToe(agent, eval_random, board_size=board_size, win_size=win_size)
        eval_game.play(n_eval_games)
        mean_reward = (eval_game.wins['DQNAgent'] - eval_game.wins['RandomPlus']) / n_eval_games
        reward_values.append(mean_reward)
        print(f"t = {str(t):5}\t reward = {round(mean_reward, 4)}\t{eval_game.wins}\n")

        torch.save(agent.state_dict(), PATH + f'model_{t}')
        torch.save(optimizer.state_dict(), PATH + f'opt_{t}')

torch.save(agent.state_dict(), PATH + f'model_{t}')
torch.save(optimizer.state_dict(), PATH + f'opt_{t}')

t = 0    	 reward = -1.0	{'DQNAgent': 0, 'RandomPlus': 100}

t = 100  	 loss = 0.0003841750440187752	 eps = 0.9987
t = 200  	 loss = 0.0013114686589688063	 eps = 0.9973
t = 300  	 loss = 0.0014353715814650059	 eps = 0.996
t = 400  	 loss = 0.0027881222777068615	 eps = 0.9947
t = 500  	 loss = 0.0027120334561914206	 eps = 0.9933
t = 500  	 reward = -0.96	{'DQNAgent': 2, 'RandomPlus': 98}

t = 600  	 loss = 0.0022848183289170265	 eps = 0.992
t = 700  	 loss = 0.0027413733769208193	 eps = 0.9907
t = 800  	 loss = 0.0028765317983925343	 eps = 0.9893
t = 900  	 loss = 0.002784157171845436	 eps = 0.988
t = 1000 	 loss = 0.002125442260876298	 eps = 0.9867
t = 1000 	 reward = -0.9	{'DQNAgent': 5, 'RandomPlus': 95}

t = 1100 	 loss = 0.002114892704412341	 eps = 0.9853
t = 1200 	 loss = 0.003842887468636036	 eps = 0.984
t = 1300 	 loss = 0.0021191625855863094	 eps = 0.9827
t = 1400 	 loss = 0.004602209664881229	 eps = 0.9813
t = 1500 	 loss = 0.0019302833825349808	 eps = 0.98
t = 1500 	 reward =

In [ ]:
agent.load_state_dict(torch.load(f'{PATH}model_45500'))
optimizer.load_state_dict(torch.load(f'{PATH}opt_45500'))

In [ ]:
# ========== ОЦЕНКА КАЧЕСТВА НА НОВОМ БОТЕ ==========


with open('out.txt', 'w') as f:
  for t in range(45_600, total_steps):
    print(f't = {t}. Ход {game.players[-1].name}', file=f)

    state = (np.zeros((board_size, board_size)), -1) # Начальное состояние игры. state = (state_2d, turn)
    turn = next_turn = -1

    while(next_turn != 0):
        current_player = game.players[turn]
        if current_player.name == 'DQNAgent':
            agent.epsilon = linear_decay(init_epsilon, final_epsilon, t, decay_steps)

        action = agent.get_action(state)
        print(action, file=f)
        next_state_2d, next_turn, reward = game.play_turn(state, action)
        state_2d, turn = state

        if next_turn == 0:
            if (reward == 0): print('Ничья!\n', file=f)
            else: print(f'Победа ({game.players[reward * turn].name})!\n', file=f)
            game.players = {-1: game.players[1], 1: game.players[-1]}

        exp_replay.add(turn * state_2d, action, reward, next_turn * next_state_2d, next_turn == 0) #state, action, reward, new_state, done

        # Обучение на минибатче
        if len(exp_replay) >= batch_size:
            states, actions, rewards, next_states, dones, indices, weights = exp_replay.sample(batch_size, augmentation=True)
            loss = compute_td_loss(states, actions, rewards, next_states, dones,
                                   agent, target_network, weights, indices, gamma, prioritized=True)
            loss.backward()
            grad_norm = nn.utils.clip_grad_norm_(agent.parameters(), max_grad_norm)
            optimizer.step()
            optimizer.zero_grad()

        state = next_state_2d, next_turn

    # Каждые refresh_target_network_freq обновляются веса target сети
    if t % refresh_target_network_freq == 0:
        target_network.load_state_dict(agent.state_dict())

    # Вывод лосса  с заданной частотой
    if t % loss_freq == 0 and t > 0:
        loss_values.append(loss)
        print(f"t = {str(t):5}\t loss = {loss}\t eps = {round(agent.epsilon, 4)}")

    # Вывод награды с заданной частотой
    if t % eval_freq == 0:
        agent.epsilon = 0
        eval_random = RandomPlus(board_size, win_size, win=True, defense=True, win_2=True, defense_2=True)
        eval_game = TicTacToe(agent, eval_random, board_size=board_size, win_size=win_size)
        eval_game.play(n_eval_games)
        mean_reward = (eval_game.wins['DQNAgent'] - eval_game.wins['RandomPlus']) / n_eval_games
        reward_values.append(mean_reward)
        print(f"t = {str(t):5}\t reward = {round(mean_reward, 4)}\t{eval_game.wins}\n")

        torch.save(agent.state_dict(), PATH + f'model_{t}')
        torch.save(optimizer.state_dict(), PATH + f'opt_{t}')

torch.save(agent.state_dict(), PATH + f'model_{t}')
torch.save(optimizer.state_dict(), PATH + f'opt_{t}')

t = 45600	 loss = None	 eps = 0.392
t = 45700	 loss = 0.0007163088885135949	 eps = 0.3907
t = 45800	 loss = 0.0023001134395599365	 eps = 0.3893
t = 45900	 loss = 0.003419699613004923	 eps = 0.388
t = 46000	 loss = 0.0007219476974569261	 eps = 0.3867
t = 46000	 reward = 0.48	{'DQNAgent': 74, 'RandomPlus': 26}

t = 46100	 loss = 0.000741938129067421	 eps = 0.3853
t = 46200	 loss = 0.00024738075444474816	 eps = 0.384
t = 46300	 loss = 0.0008657713187858462	 eps = 0.3827
t = 46400	 loss = 0.00047062651719897985	 eps = 0.3813
t = 46500	 loss = 0.0006787946331314743	 eps = 0.38
t = 46500	 reward = 0.66	{'DQNAgent': 82, 'RandomPlus': 16}

t = 46600	 loss = 0.0011206482304260135	 eps = 0.3787
t = 46700	 loss = 0.0007954401080496609	 eps = 0.3773
t = 46800	 loss = 0.0005883672856725752	 eps = 0.376
t = 46900	 loss = 0.0005571823567152023	 eps = 0.3747
t = 47000	 loss = 0.0006007581250742078	 eps = 0.3733
t = 47000	 reward = 0.49	{'DQNAgent': 73, 'RandomPlus': 24}

t = 47100	 loss = 0.0010953887

In [ ]:
agent.load_state_dict(torch.load(f'{PATH}model_49000'))
optimizer.load_state_dict(torch.load(f'{PATH}opt_49000'))

main_random = RandomPlus(board_size, win_size, win=True)
game = TicTacToe(agent, main_random, board_size=board_size, win_size=win_size)

In [ ]:
# ========== CONTINUE ==========


with open('out.txt', 'w') as f:
  for t in range(49_100, total_steps):
    print(f't = {t}. Ход {game.players[-1].name}', file=f)

    state = (np.zeros((board_size, board_size)), -1) # Начальное состояние игры. state = (state_2d, turn)
    turn = next_turn = -1

    while(next_turn != 0):
        current_player = game.players[turn]
        if current_player.name == 'DQNAgent':
            agent.epsilon = linear_decay(init_epsilon, final_epsilon, t, decay_steps)

        action = agent.get_action(state)
        print(action, file=f)
        next_state_2d, next_turn, reward = game.play_turn(state, action)
        state_2d, turn = state

        if next_turn == 0:
            if (reward == 0): print('Ничья!\n', file=f)
            else: print(f'Победа ({game.players[reward * turn].name})!\n', file=f)
            game.players = {-1: game.players[1], 1: game.players[-1]}

        exp_replay.add(turn * state_2d, action, reward, next_turn * next_state_2d, next_turn == 0) #state, action, reward, new_state, done

        # Обучение на минибатче
        if len(exp_replay) >= batch_size:
            states, actions, rewards, next_states, dones, indices, weights = exp_replay.sample(batch_size, augmentation=True)
            loss = compute_td_loss(states, actions, rewards, next_states, dones,
                                   agent, target_network, weights, indices, gamma, prioritized=True)
            loss.backward()
            grad_norm = nn.utils.clip_grad_norm_(agent.parameters(), max_grad_norm)
            optimizer.step()
            optimizer.zero_grad()

        state = next_state_2d, next_turn

    # Каждые refresh_target_network_freq обновляются веса target сети
    if t % refresh_target_network_freq == 0:
        target_network.load_state_dict(agent.state_dict())

    # Вывод лосса  с заданной частотой
    if t % loss_freq == 0 and t > 0:
        loss_values.append(loss)
        print(f"t = {str(t):5}\t loss = {loss}\t eps = {round(agent.epsilon, 4)}")

    # Вывод награды с заданной частотой
    if t % eval_freq == 0:
        agent.epsilon = 0
        eval_random = RandomPlus(board_size, win_size, win=True, defense=True, win_2=True, defense_2=True)
        eval_game = TicTacToe(agent, eval_random, board_size=board_size, win_size=win_size)
        eval_game.play(n_eval_games)
        mean_reward = (eval_game.wins['DQNAgent'] - eval_game.wins['RandomPlus']) / n_eval_games
        reward_values.append(mean_reward)
        print(f"t = {str(t):5}\t reward = {round(mean_reward, 4)}\t{eval_game.wins}\n")

        torch.save(agent.state_dict(), PATH + f'model_{t}')
        torch.save(optimizer.state_dict(), PATH + f'opt_{t}')

torch.save(agent.state_dict(), PATH + f'model_{t}')
torch.save(optimizer.state_dict(), PATH + f'opt_{t}')

t = 49100	 loss = None	 eps = 0.3862
t = 49200	 loss = 0.0006231703446246684	 eps = 0.385
t = 49300	 loss = 0.0008477765950374305	 eps = 0.3838
t = 49400	 loss = 0.00030077152769081295	 eps = 0.3825
t = 49500	 loss = 0.0006357431411743164	 eps = 0.3812
t = 49500	 reward = 0.7	{'DQNAgent': 82, 'RandomPlus': 12}

t = 49600	 loss = 0.0006129497196525335	 eps = 0.38
t = 49700	 loss = 0.0012368080206215382	 eps = 0.3788
t = 49800	 loss = 0.0006861520814709365	 eps = 0.3775
t = 49900	 loss = 0.0003389932098798454	 eps = 0.3762
t = 50000	 loss = 0.0006419041892513633	 eps = 0.375
t = 50000	 reward = 0.72	{'DQNAgent': 86, 'RandomPlus': 14}

t = 50100	 loss = 0.0004673107177950442	 eps = 0.3738
t = 50200	 loss = 0.00036452687345445156	 eps = 0.3725
t = 50300	 loss = 0.0010725996689870954	 eps = 0.3712
t = 50400	 loss = 0.0014924638671800494	 eps = 0.37
t = 50500	 loss = 0.0010145039996132255	 eps = 0.3688
t = 50500	 reward = 0.57	{'DQNAgent': 76, 'RandomPlus': 19}



KeyboardInterrupt: 

In [ ]:
# ========== CONTINUE ==========


with open('out.txt', 'w') as f:
  for t in range(50_600, total_steps):
    print(f't = {t}. Ход {game.players[-1].name}', file=f)

    state = (np.zeros((board_size, board_size)), -1) # Начальное состояние игры. state = (state_2d, turn)
    turn = next_turn = -1

    while(next_turn != 0):
        current_player = game.players[turn]
        if current_player.name == 'DQNAgent':
            agent.epsilon = linear_decay(init_epsilon, final_epsilon, t, decay_steps)

        action = agent.get_action(state)
        print(action, file=f)
        next_state_2d, next_turn, reward = game.play_turn(state, action)
        state_2d, turn = state

        if next_turn == 0:
            if (reward == 0): print('Ничья!\n', file=f)
            else: print(f'Победа ({game.players[reward * turn].name})!\n', file=f)
            game.players = {-1: game.players[1], 1: game.players[-1]}

        exp_replay.add(turn * state_2d, action, reward, next_turn * next_state_2d, next_turn == 0) #state, action, reward, new_state, done

        # Обучение на минибатче
        if len(exp_replay) >= batch_size:
            states, actions, rewards, next_states, dones, indices, weights = exp_replay.sample(batch_size, augmentation=True)
            loss = compute_td_loss(states, actions, rewards, next_states, dones,
                                   agent, target_network, weights, indices, gamma, prioritized=True)
            loss.backward()
            grad_norm = nn.utils.clip_grad_norm_(agent.parameters(), max_grad_norm)
            optimizer.step()
            optimizer.zero_grad()

        state = next_state_2d, next_turn

    # Каждые refresh_target_network_freq обновляются веса target сети
    if t % refresh_target_network_freq == 0:
        target_network.load_state_dict(agent.state_dict())

    # Вывод лосса  с заданной частотой
    if t % loss_freq == 0 and t > 0:
        loss_values.append(loss)
        print(f"t = {str(t):5}\t loss = {loss}\t eps = {round(agent.epsilon, 4)}")

    # Вывод награды с заданной частотой
    if t % eval_freq == 0:
        agent.epsilon = 0
        eval_random = RandomPlus(board_size, win_size, win=True, defense=True, win_2=True, defense_2=True)
        eval_game = TicTacToe(agent, eval_random, board_size=board_size, win_size=win_size)
        eval_game.play(n_eval_games)
        mean_reward = (eval_game.wins['DQNAgent'] - eval_game.wins['RandomPlus']) / n_eval_games
        reward_values.append(mean_reward)
        print(f"t = {str(t):5}\t reward = {round(mean_reward, 4)}\t{eval_game.wins}\n")

        torch.save(agent.state_dict(), PATH + f'model_{t}')
        torch.save(optimizer.state_dict(), PATH + f'opt_{t}')

torch.save(agent.state_dict(), PATH + f'model_{t}')
torch.save(optimizer.state_dict(), PATH + f'opt_{t}')

t = 50600	 loss = 0.0005726382369175553	 eps = 0.3675
t = 50700	 loss = 0.0007521308725699782	 eps = 0.3662
t = 50800	 loss = 0.00040385156171396375	 eps = 0.365
t = 50900	 loss = 0.0011169458739459515	 eps = 0.3638
t = 51000	 loss = 0.0002975226962007582	 eps = 0.3625
t = 51000	 reward = 0.51	{'DQNAgent': 75, 'RandomPlus': 24}

t = 51100	 loss = 0.0004504479584284127	 eps = 0.3612
t = 51200	 loss = 0.0026520220562815666	 eps = 0.36
t = 51300	 loss = 0.00029461923986673355	 eps = 0.3588
t = 51400	 loss = 0.0013692324282601476	 eps = 0.3575
t = 51500	 loss = 0.00035188253968954086	 eps = 0.3562
t = 51500	 reward = 0.64	{'DQNAgent': 81, 'RandomPlus': 17}

t = 51600	 loss = 0.0012226385297253728	 eps = 0.355
t = 51700	 loss = 0.00028253940399736166	 eps = 0.3538
t = 51800	 loss = 0.000478589441627264	 eps = 0.3525
t = 51900	 loss = 0.0004016034654341638	 eps = 0.3512
t = 52000	 loss = 0.0006413225783035159	 eps = 0.35
t = 52000	 reward = 0.77	{'DQNAgent': 87, 'RandomPlus': 10}

t = 52100	

In [ ]:
agent.load_state_dict(torch.load(f'{PATH}model_63500'))
optimizer.load_state_dict(torch.load(f'{PATH}opt_63500'))

main_random = RandomPlus(board_size, win_size, win=True)
game = TicTacToe(agent, main_random, board_size=board_size, win_size=win_size)

In [ ]:
# ========== CONTINUE ==========


with open('out.txt', 'w') as f:
  for t in range(63_600, total_steps):
    print(f't = {t}. Ход {game.players[-1].name}', file=f)

    state = (np.zeros((board_size, board_size)), -1) # Начальное состояние игры. state = (state_2d, turn)
    turn = next_turn = -1

    while(next_turn != 0):
        current_player = game.players[turn]
        if current_player.name == 'DQNAgent':
            agent.epsilon = linear_decay(init_epsilon, final_epsilon, t, decay_steps)

        action = agent.get_action(state)
        print(action, file=f)
        next_state_2d, next_turn, reward = game.play_turn(state, action)
        state_2d, turn = state

        if next_turn == 0:
            if (reward == 0): print('Ничья!\n', file=f)
            else: print(f'Победа ({game.players[reward * turn].name})!\n', file=f)
            game.players = {-1: game.players[1], 1: game.players[-1]}

        exp_replay.add(turn * state_2d, action, reward, next_turn * next_state_2d, next_turn == 0) #state, action, reward, new_state, done

        # Обучение на минибатче
        if len(exp_replay) >= batch_size:
            states, actions, rewards, next_states, dones, indices, weights = exp_replay.sample(batch_size, augmentation=True)
            loss = compute_td_loss(states, actions, rewards, next_states, dones,
                                   agent, target_network, weights, indices, gamma, prioritized=True)
            loss.backward()
            grad_norm = nn.utils.clip_grad_norm_(agent.parameters(), max_grad_norm)
            optimizer.step()
            optimizer.zero_grad()

        state = next_state_2d, next_turn

    # Каждые refresh_target_network_freq обновляются веса target сети
    if t % refresh_target_network_freq == 0:
        target_network.load_state_dict(agent.state_dict())

    # Вывод лосса  с заданной частотой
    if t % loss_freq == 0 and t > 0:
        loss_values.append(loss)
        print(f"t = {str(t):5}\t loss = {loss}\t eps = {round(agent.epsilon, 4)}")

    # Вывод награды с заданной частотой
    if t % eval_freq == 0:
        agent.epsilon = 0
        eval_random = RandomPlus(board_size, win_size, win=True, defense=True, win_2=True, defense_2=True)
        eval_game = TicTacToe(agent, eval_random, board_size=board_size, win_size=win_size)
        eval_game.play(n_eval_games)
        mean_reward = (eval_game.wins['DQNAgent'] - eval_game.wins['RandomPlus']) / n_eval_games
        reward_values.append(mean_reward)
        print(f"t = {str(t):5}\t reward = {round(mean_reward, 4)}\t{eval_game.wins}\n")

        torch.save(agent.state_dict(), PATH + f'model_{t}')
        torch.save(optimizer.state_dict(), PATH + f'opt_{t}')

torch.save(agent.state_dict(), PATH + f'model_{t}')
torch.save(optimizer.state_dict(), PATH + f'opt_{t}')

t = 63600	 loss = 0.009547475725412369	 eps = 0.25
t = 63700	 loss = 0.0023229101207107306	 eps = 0.25
t = 63800	 loss = 0.0004563727416098118	 eps = 0.25
t = 63900	 loss = 0.0004655389639083296	 eps = 0.25
t = 64000	 loss = 0.0006297405925579369	 eps = 0.25
t = 64000	 reward = 0.48	{'DQNAgent': 72, 'RandomPlus': 24}

t = 64100	 loss = 0.0004901965730823576	 eps = 0.25
t = 64200	 loss = 0.0008226131903938949	 eps = 0.25
t = 64300	 loss = 0.0005874842754565179	 eps = 0.25
t = 64400	 loss = 0.0003001315053552389	 eps = 0.25
t = 64500	 loss = 0.0023031188175082207	 eps = 0.25
t = 64500	 reward = 0.53	{'DQNAgent': 72, 'RandomPlus': 19}

t = 64600	 loss = 0.0005634687258861959	 eps = 0.25
t = 64700	 loss = 0.0016298370901495218	 eps = 0.25
t = 64800	 loss = 0.0012274766340851784	 eps = 0.25
t = 64900	 loss = 0.0007745488546788692	 eps = 0.25
t = 65000	 loss = 0.0008624959737062454	 eps = 0.25
t = 65000	 reward = 0.48	{'DQNAgent': 73, 'RandomPlus': 25}

t = 65100	 loss = 0.000526983931194990

KeyboardInterrupt: 

#Тестирование обученных моделей (инференс с маскированием)

In [ ]:
PATH = '/content/drive/MyDrive/TicTacToe_9/'

In [ ]:
# Сравнение обученных моделей (START -> CONTINUE)
eval_random = RandomPlus(board_size, win_size, win=True, defense=True, win_2=True, defense_2=True)
agent.epsilon = 0
agent.masking = True

for i in range(0, 49_001, 500):
    agent.load_state_dict(torch.load(f'{PATH}model_{i}', map_location=torch.device('cpu')))
    eval_game = TicTacToe(agent, eval_random, board_size=board_size, win_size=win_size)
    eval_game.play(1000)
    print(f'{i:5}', eval_game.wins)


    0 {'DQNAgent': 3, 'RandomPlus': 997}
  500 {'DQNAgent': 10, 'RandomPlus': 979}
 1000 {'DQNAgent': 32, 'RandomPlus': 951}
 1500 {'DQNAgent': 66, 'RandomPlus': 885}
 2000 {'DQNAgent': 105, 'RandomPlus': 862}
 2500 {'DQNAgent': 110, 'RandomPlus': 861}
 3000 {'DQNAgent': 155, 'RandomPlus': 824}
 3500 {'DQNAgent': 227, 'RandomPlus': 753}
 4000 {'DQNAgent': 203, 'RandomPlus': 760}
 4500 {'DQNAgent': 151, 'RandomPlus': 798}
 5000 {'DQNAgent': 177, 'RandomPlus': 752}
 5500 {'DQNAgent': 234, 'RandomPlus': 693}
 6000 {'DQNAgent': 284, 'RandomPlus': 677}
 6500 {'DQNAgent': 262, 'RandomPlus': 671}
 7000 {'DQNAgent': 223, 'RandomPlus': 701}
 7500 {'DQNAgent': 279, 'RandomPlus': 665}
 8000 {'DQNAgent': 275, 'RandomPlus': 649}
 8500 {'DQNAgent': 271, 'RandomPlus': 645}
 9000 {'DQNAgent': 339, 'RandomPlus': 591}
 9500 {'DQNAgent': 297, 'RandomPlus': 605}
10000 {'DQNAgent': 265, 'RandomPlus': 636}
10500 {'DQNAgent': 289, 'RandomPlus': 629}
11000 {'DQNAgent': 367, 'RandomPlus': 561}
11500 {'DQNAgen

In [ ]:
# Сравнение обученных моделей (SELF PLAY - BAD)
eval_random = RandomPlus(board_size, win_size, win=True, defense=True, win_2=True, defense_2=True)
agent.epsilon = 0
agent.masking = True

for i in range(49_500, 82_501, 500):
    agent.load_state_dict(torch.load(f'{PATH}model_{i}', map_location=torch.device('cpu')))
    eval_game = TicTacToe(agent, eval_random, board_size=board_size, win_size=win_size)
    eval_game.play(1000)
    print(f'{i:5}', eval_game.wins)

49500 {'DQNAgent': 781, 'RandomPlus': 175}
50000 {'DQNAgent': 774, 'RandomPlus': 171}
50500 {'DQNAgent': 779, 'RandomPlus': 171}
51000 {'DQNAgent': 786, 'RandomPlus': 155}
51500 {'DQNAgent': 771, 'RandomPlus': 177}
52000 {'DQNAgent': 794, 'RandomPlus': 165}
52500 {'DQNAgent': 823, 'RandomPlus': 140}
53000 {'DQNAgent': 801, 'RandomPlus': 162}
53500 {'DQNAgent': 810, 'RandomPlus': 147}
54000 {'DQNAgent': 773, 'RandomPlus': 188}
54500 {'DQNAgent': 782, 'RandomPlus': 166}
55000 {'DQNAgent': 723, 'RandomPlus': 228}
55500 {'DQNAgent': 747, 'RandomPlus': 215}
56000 {'DQNAgent': 712, 'RandomPlus': 228}
56500 {'DQNAgent': 798, 'RandomPlus': 163}
57000 {'DQNAgent': 774, 'RandomPlus': 173}
57500 {'DQNAgent': 791, 'RandomPlus': 165}
58000 {'DQNAgent': 735, 'RandomPlus': 219}
58500 {'DQNAgent': 758, 'RandomPlus': 197}
59000 {'DQNAgent': 748, 'RandomPlus': 210}
59500 {'DQNAgent': 764, 'RandomPlus': 202}
60000 {'DQNAgent': 746, 'RandomPlus': 204}
60500 {'DQNAgent': 727, 'RandomPlus': 205}
61000 {'DQN

In [ ]:
# Сравнение лучших моделей (без проигрышей)
models = [...]

agent.epsilon = 0
agent.masking = True

for model in models:
    agent.load_state_dict(torch.load(f'{PATH}model_{model}', map_location=torch.device('cpu')))
    eval_game = TicTacToe(agent, eval_random, board_size=board_size, win_size=win_size)
    eval_game.play(10_000)
    print(model, eval_game.wins)

In [ ]:
# Загрузка самой лучшей модели
agent.load_state_dict(torch.load(f'{PATH}model_45500', map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
agent.epsilon = 0
test_game = TicTacToe(agent, Human(), board_size=board_size, win_size=win_size)
test_game.play(4, True)
test_game.wins

player -1's turn:
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
player -1's turn:
 .  .  .  .  .  .  .
 .  .  .  .  .  .  .
 .  .  .  .  .  .  .
 .  .  .  X  .  .  .
 .  .  .  .  .  .  .
 .  .  .  .  .  .  .
 .  .  .  .  .  .  .
Введите ваш ход (Строка, столбец)
3 3
player 1's turn:
 .  .  .  .  .  .  .
 .  .  .  .  .  .  .
 .  .  O  .  .  .  .
 .  .  .  X  .  .  .
 .  .  .  .  .  .  .
 .  .  .  .  .  .  .
 .  .  .  .  .  .  .
player -1's turn:
 .  .  .  .  .  .  .
 .  .  .  .  .  .  .
 .  .  O  X  .  .  .
 .  .  .  X  .  .  .
 .  .  .  .  .  .  .
 .  .  .  .  .  .  .
 .  .  .  .  .  .  .
Введите ваш ход (Строка, столбец)
2 4
player 1's turn:
 .  .  .  .  .  .  .
 .  .  .  O  .  .  .
 .  .  O  X  .  .  .
 .  .  .  X  .  .  .
 .  .  .  .  .  .  .
 .  .  .  .  .  .  .
 .  .  .  .  .  .  .
player -1's turn:
 .  .  .  .  .  .  .
 .  .  .  O  .  .  .
 .  .  O  X  .  .  .
 .  .  .  X  X  .  .
 .  .  .  .  .  .  .
 .  .  .  .  .  .  .
 .  . 

{'DQNAgent': 2, 'Human': 2}

# Первый ход за крестики и значения $Q$-фунцкии в начальном состоянии

In [ ]:
state2d = torch.tensor(np.zeros((1, 7, 7))).to(device)

q_values = agent(state2d).squeeze(0).detach().cpu().numpy()
np.unravel_index(q_values.argmax(), q_values.shape)

(3, 3)

In [ ]:
q_values.round(4)

array([[-0.0684, -0.064 , -0.0358, -0.0494, -0.0263, -0.0711, -0.0195],
       [-0.0457, -0.0207, -0.0132, -0.0035, -0.0091, -0.029 , -0.0438],
       [-0.0156, -0.0113,  0.0434,  0.0295,  0.0367, -0.0051, -0.0104],
       [-0.0425, -0.0255,  0.0354,  0.0539,  0.0183, -0.0157, -0.0361],
       [-0.0721, -0.0236,  0.0205,  0.0216,  0.0134, -0.0366, -0.0267],
       [-0.0979, -0.0705, -0.0431, -0.0291, -0.044 , -0.0472, -0.0386],
       [-0.0893, -0.0818, -0.0544, -0.0399, -0.0268, -0.0384, -0.0467]],
      dtype=float32)